v1 meta store product availability checker

includes:
- non-replacement part accessories

excludes:
- replacement part accessories
- meta warranties
- meta gift cards
- flagship products (which require click to add-to-cart to see availability)

caveats:
- meta quest 3 facial interface & head strap (elemental blue & blood orange) variations combined into one availability

future improvements:
- check all regions
- check replacement part availability

In [ ]:
# text corpus availability check

##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

cleaned = []
url = 'https://www.meta.com/jp/quest/products/quest-2/'

driver = webdriver.Chrome()  # bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
text = soup.get_text().lower()

print(text)

In [117]:
# get text corpus

##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

cleaned = []
product_list = ['d link vr air bridge',
                'link cable',
                'quest 2 active pack',
                'quest 2 anker charging dock',
                'quest 2 carrying case',
                'quest 2 elite strap',
                'quest 2 elite strap battery',
                'quest 2 fit pack',
                'quest 2 logitech G333 vr gaming earphones',
                'quest 2 logitech pro gaming headset',
                'quest 2 logitech vr off ear audio',
                'quest 2 virtuclear lens inserts',
                'quest 2 vr cover facial interface foam replacement set',
                'quest 3 carrying case',
                'quest 3 charging dock',
                'quest 3 elite strap',
                'quest 3 elite strap battery',
                'quest 3 facial interface head strap',
                'quest 3 helloreal silicone cover',
                'quest 3 razer hammerhead hyperspeed earbuds',
                'quest 3 silicone facial interface',
                'quest 3 zenni vr prescription lenses',
                'quest active straps touch plus',
                'quest pro carry case',
                'quest pro compact charging dock and adapter',
                'quest pro full light blocker',
                'quest pro vr earphones',
                'quest touch pro controllers and charging dock',
                'razer adjustable head strap system meta quest 2',
                'razer facial interface meta quest 2',
                'soundcore anker wireless vr earbuds']

for product in product_list:
    url_completion = product.replace(' ', '-')
    url = 'https://www.meta.com/quest/accessories/{}'.format(url_completion)

    # bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    text = soup.get_text().lower().replace('\n', '')

    # account for different product names in metadata
    # product is the url name
    # product_pattern is the product metadata name
    if product == 'd link vr air bridge':
        product_pattern = 'd-link vr air bridge wifi adapter'
    elif product == 'quest 2 anker charging dock':
        product_pattern = 'anker charging dock for meta quest 2'
    elif product == 'quest 2 elite strap battery':
        product_pattern = 'meta quest 2 elite strap with battery'
    elif product == 'quest 2 logitech G333 vr gaming earphones':
        product_pattern = 'logitech g333 vr gaming earphones for quest 2'
    elif product == 'quest 2 logitech pro gaming headset':
        product_pattern = 'logitech g pro gaming headset for meta quest 2'
    elif product == 'quest 2 logitech vr off ear audio':
        product_pattern = 'logitech off-ear vr headphones for meta quest 2'
    elif product == 'quest 2 virtuclear lens inserts':
        product_pattern = 'virtuclear lens insert'
    elif product == 'quest 2 vr cover facial interface foam replacement set':
        product_pattern = 'meta quest 2 vr cover facial interface & foam replacement set'
    elif product == 'quest 3 elite strap battery':
        product_pattern = 'quest 3 meta quest 3 elite strap with battery'
    elif product == 'quest 3 facial interface head strap':
        product_pattern = 'meta quest 3 facial interface & head strap (elemental blue & blood orange)'
    elif product == 'quest 3 helloreal silicone cover':
        product_pattern = 'helloreal silicone cover for meta quest 3'
    elif product == 'quest 3 zenni vr prescription lenses':
        product_pattern = 'zenni® vr prescription lenses for meta quest 3'
    elif product == 'quest active straps touch plus':
        product_pattern = 'quest 3 meta quest active straps (for touch plus controllers)'
    elif product == 'quest pro carry case':
        product_pattern = 'meta quest pro carrying case'
    elif product == 'quest pro compact charging dock and adapter':
        product_pattern = 'meta quest pro compact charging dock'
    elif product == 'quest touch pro controllers and charging dock':
        product_pattern = 'meta quest touch pro vr controllers'
    elif product == 'razer adjustable head strap system meta quest 2':
        product_pattern = 'razer adjustable head strap system for quest 2'
    elif product == 'razer facial interface meta quest 2':
        product_pattern = 'razer facial interface for meta quest 2'
    elif product == 'soundcore anker wireless vr earbuds':
        product_pattern = 'anker soundcore vr wireless earbuds'
    else:
        product_pattern = product

    pattern = '{} (.*?) usd'.format(re.escape(product_pattern))
    match = re.search(pattern, text)
    match_text = match.group()

    # column separator prep for pandas table
    replacement_key = product.split()[-1]
    if '$' in match_text:
        cleaned.append(match_text.replace('$', '`$').replace(
            replacement_key, replacement_key + '`').replace('notify me', '`notify me'))

print(cleaned)

['d-link vr air bridge` wifi adapter | meta quest | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsbuy nowadd to bag                   made for meta                                d-link vr air bridge`                `$99 usd', 'link cable` | connect pc to vr headset | meta quest | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specs  add to bag`$79.99 usd', 'quest 2 active pack` | vr accessories | meta quest | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsbuy nowadd to bag                    meta quest 2 active pack`                  `$69.99 usd', 'anker charging dock` for meta quest 2 | meta quest | meta store    free delivery        extended ret

In [215]:
# text corpus cleaning

import numpy as np

df = pd.DataFrame(cleaned) #create dataframe
df1 = df[0].str.split('`', expand=True) #use '`' as column separator
df2 = df1.sort_values(by=[0])

def get_price(row):
    last_valid_index = row.last_valid_index()
    return row[last_valid_index] if last_valid_index is not None else np.nan

df2['price'] = df2.apply(get_price, axis=1)

# Define a custom function to find the second-to-last non-null column
def find_second_to_last_non_null_values(row):
    non_null_columns = [col for col in reversed(df2.columns) if not pd.isna(row[col])]
    return row[non_null_columns[2]] if len(non_null_columns) >= 2 else np.nan

# Apply the custom function along rows
df2['second_to_last_non_null'] = df2.apply(find_second_to_last_non_null_values, axis=1)

df3 = df2[[0, 'second_to_last_non_null', 'price']]
df3.columns = ['product', 'availability', 'price'] # add column titles
df3['availability'] = df3['availability'].str.replace('notify me', 'out of stock')
df3['availability'] = df3['availability'].replace({'^(?!out of stock$).*': 'available'}, regex=True)
display(df3)

C:\Users\yang4\AppData\Local\Temp\ipykernel_36024\656897689.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['availability'] = df3['availability'].str.replace('notify me', 'out of stock')
C:\Users\yang4\AppData\Local\Temp\ipykernel_36024\656897689.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['availability'] = df3['availability'].replace({'^(?!out of stock$).*': 'available'}, regex=True)


,product,availability,price
3,anker charging dock,available,$99 usd
30,anker soundcore vr wireless earbuds,available,$99.99 usd
0,d-link vr air bridge,available,$99 usd
18,helloreal silicone cover,available,$69.99 usd
1,link cable,available,$79.99 usd
9,logitech g pro gaming headset,available,$100 usd
8,logitech g333 vr gaming earphones,available,$50 usd
10,logitech off-ear vr headphones for meta quest ...,available,$99.99 usd
6,meta quest 2 elite strap with battery,available,$119.99 usd
12,meta quest 2 vr cover facial interface & foam ...,available,$29 usd
